In [1]:
# Dependencies and Setup

import json
import pandas as pd
import requests
import numpy as py
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, inspect, func, distinct
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3
from sqlite3 import Error

In [2]:
# Create engine and engine connection to SQLite Database

engine = create_engine("sqlite:///../SqLIte_files/NBAfantasyML.sqlite")
conn = engine.connect()

In [3]:
# Create function to try and connect to a sqlite database.  if it doesn't work, create one...

def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()

In [4]:
# If connection succeeds

if __name__ == '__main__':
    create_connection("../NBAfantasyML.sqlite")

2.6.0


In [5]:
#Bring in data from internet as object then convert ot Json

schedule_response_obj=requests.get("http://data.nba.net/prod/v1/2018/schedule.json")
schedule_response=schedule_response_obj.json()

In [6]:
# Count the number of Games for the season 

games_response_count=schedule_response["league"]["standard"]
game_number=len(games_response_count)
# game_number

In [7]:
###    Elements for URL, append List, create DataFrame  ###

# Bring in Game Id and GameData ("startDataEastern") elements from NBA Schedule later used to create box_score URL
# Store elements in list
# Convert list to DataFrame and modify datatypes

# Get elements
NBA_Schedule=[]
for item in schedule_response["league"]["standard"]:
    item_dict={}
    item_dict["gameId"]=item["gameId"]
    item_dict["startDateEastern"]=item["startDateEastern"]

# Append List
    NBA_Schedule.append({"gameId":item_dict["gameId"],
                         "startDataEastern":item_dict["startDateEastern"]})

# Convert NBA_Schedule=[] list to DataFrame and modify Datatypes
NBA_Schedule_DF_initial=pd.DataFrame(NBA_Schedule)
NBA_Schedule_DF_initial["gameId"]=NBA_Schedule_DF_initial["gameId"].astype(int)
NBA_Schedule_DF_initial["startDataEastern"]=NBA_Schedule_DF_initial["startDataEastern"].astype(int)

NBA_Schedule_DF_initial.head()  
# regular season starts October 16, 2018


,gameId,startDataEastern
0,11800001,20180928
1,11800002,20180928
2,11800003,20180929
3,11800004,20180929
4,11800005,20180929


In [20]:
# count the number of gamesId records

game_count=len(NBA_Schedule_DF_initial["gameId"])


1309

In [33]:
# # Generate Box_Score_URL and append to List Box_Score_URL_List

Box_score_URL_List =[{}]

# Looping through gameids 
#for i in range (0,game_count):
    
chosen=0

    # Generate Box Score URL using GameId and GameData ("startDataEastern")
Box_score_URL=F"http://data.nba.net/prod/v1/{NBA_Schedule_DF_initial['startDataEastern'][chosen]}/00{NBA_Schedule_DF_initial['gameId'][chosen]}_boxscore.json"

NBA_Schedule_DF_initial["boxScoreURL"]=Box_score_URL
NBA_Schedule_DF_initial["boxScoreURL"]=NBA_Schedule_DF_initial["boxScoreURL"].astype(str)
# NBA_Schedule_DF_initial.head()

,gameId,startDataEastern,boxScoreURL
0,11800001,20180928,http://data.nba.net/prod/v1/20180928/001180000...
1,11800002,20180928,http://data.nba.net/prod/v1/20180928/001180000...
2,11800003,20180929,http://data.nba.net/prod/v1/20180928/001180000...
3,11800004,20180929,http://data.nba.net/prod/v1/20180928/001180000...
4,11800005,20180929,http://data.nba.net/prod/v1/20180928/001180000...


In [11]:
# Box Score Request and request Object 

NBA_Schedule_DF_initial.boxScoreURL[chosen]
box_score_obj=requests.get(Box_score_URL)
box_score_response=box_score_obj.json()
# box_score_response

In [31]:
# Extract basic box score data EXCLUDING: INDIVIDUAL PLAYER STATS (two loops, vteam & hteam)
Team_boxscores=[]
# Vteam
gameid=box_score_response["basicGameData"]["gameId"]
teamId=box_score_response["basicGameData"]["vTeam"]["teamId"]
fastBreakPoints=box_score_response["stats"]["vTeam"]["fastBreakPoints"]
pointsInPaint=box_score_response["stats"]["vTeam"]["pointsInPaint"]
secondChancePoints=box_score_response["stats"]["vTeam"]["secondChancePoints"]
pointsOffTurnovers=box_score_response["stats"]["vTeam"]["pointsOffTurnovers"]
points=box_score_response["stats"]["vTeam"]["totals"]["points"]
fgm=box_score_response["stats"]["vTeam"]["totals"]["fgm"]
fga=box_score_response["stats"]["vTeam"]["totals"]["fga"]
fgp=box_score_response["stats"]["vTeam"]["totals"]["fgp"]
ftm=box_score_response["stats"]["vTeam"]["totals"]["ftm"]
fta=box_score_response["stats"]["vTeam"]["totals"]["fta"]
ftp=box_score_response["stats"]["vTeam"]["totals"]["ftp"]
ttm=box_score_response["stats"]["vTeam"]["totals"]["ttm"]
tta=box_score_response["stats"]["vTeam"]["totals"]["tta"]
ttp=box_score_response["stats"]["vTeam"]["totals"]["ttp"]
offReb=box_score_response["stats"]["vTeam"]["totals"]["offReb"]
defReb=box_score_response["stats"]["vTeam"]["totals"]["defReb"]
totReb=box_score_response["stats"]["vTeam"]["totals"]["totReb"]
assists=box_score_response["stats"]["vTeam"]["totals"]["assists"]
pFouls=box_score_response["stats"]["vTeam"]["totals"]["pFouls"]
steals=box_score_response["stats"]["vTeam"]["totals"]["steals"]
turnovers=box_score_response["stats"]["vTeam"]["totals"]["turnovers"]
blocks=box_score_response["stats"]["vTeam"]["totals"]["blocks"]
plusMinus=box_score_response["stats"]["vTeam"]["totals"]["plusMinus"]
min=box_score_response["stats"]["vTeam"]["totals"]["min"]
# for non_player_stats in box_score_response["basicGameData"]


31


In [ ]:
# Hteam
home_team=box_score_response["basicGameData"]["hTeam"]["teamId"]

In [37]:
# Extract ONLY player stats from box score
boxscore_player_list=[]
for boxscore_playerstats in box_score_response["stats"]["activePlayers"]:
    boxscore_player_dict={}
    boxscore_player_dict["personId"]=boxscore_playerstats["personId"]
    boxscore_player_dict["teamId"]=boxscore_playerstats["teamId"]
    boxscore_player_dict["points"]=boxscore_playerstats["points"]
    boxscore_player_dict["pos"]=boxscore_playerstats["pos"]
    boxscore_player_dict["min"]=boxscore_playerstats["min"]
    boxscore_player_dict["points"]=boxscore_playerstats["points"]
    boxscore_player_dict["fgm"]=boxscore_playerstats["fgm"]
    boxscore_player_dict["min"]=boxscore_playerstats["min"]
    boxscore_player_dict["points"]=boxscore_playerstats["points"]
    boxscore_player_dict["fgm"]=boxscore_playerstats["fgm"]
    boxscore_player_dict["fga"]=boxscore_playerstats["fga"]
    boxscore_player_dict["fgp"]=boxscore_playerstats["fgp"]
    boxscore_player_dict["ftm"]=boxscore_playerstats["ftm"]
    boxscore_player_dict["fta"]=boxscore_playerstats["fta"]
    boxscore_player_dict["ftp"]=boxscore_playerstats["ftp"]
    boxscore_player_dict["tpm"]=boxscore_playerstats["tpm"]
    boxscore_player_dict["tpa"]=boxscore_playerstats["tpa"]
    boxscore_player_dict["tpp"]=boxscore_playerstats["tpp"]
    boxscore_player_dict["offReb"]=boxscore_playerstats["offReb"]
    boxscore_player_dict["defReb"]=boxscore_playerstats["defReb"]
    boxscore_player_dict["totReb"]=boxscore_playerstats["totReb"]
    boxscore_player_dict["assists"]=boxscore_playerstats["assists"]
    boxscore_player_dict["pFouls"]=boxscore_playerstats["pFouls"]
    boxscore_player_dict["steals"]=boxscore_playerstats["steals"]
    boxscore_player_dict["turnovers"]=boxscore_playerstats["totReb"]
    boxscore_player_dict["blocks"]=boxscore_playerstats["blocks"]
    boxscore_player_dict["plusMinus"]=boxscore_playerstats["plusMinus"]
    boxscore_player_dict["dnp"]=boxscore_playerstats["dnp"]
    
    boxscore_player_list.append({"personId":boxscore_playerstats["personId"],
                                "teamId":boxscore_playerstats["teamId"],
                                "points":boxscore_playerstats["points"],
                                "pos":boxscore_playerstats["pos"],
                                "min":boxscore_playerstats["min"],
                                "points":boxscore_playerstats["points"],
                                "fgm":boxscore_playerstats["fgm"],
                                "min":boxscore_playerstats["min"],
                                "points":boxscore_playerstats["points"],
                                "fgm":boxscore_playerstats["fgm"],
                                "fga":boxscore_playerstats["fga"],
                                "fgp":boxscore_playerstats["fgp"],
                                "ftm":boxscore_playerstats["ftm"],
                                "fta":boxscore_playerstats["fta"],
                                "ftp":boxscore_playerstats["ftp"],
                                "tpm":boxscore_playerstats["tpm"],
                                "tpa":boxscore_playerstats["tpa"],
                                "tpp":boxscore_playerstats["tpp"],
                                "offReb":boxscore_playerstats["offReb"],
                                "defReb":boxscore_playerstats["defReb"],
                                "totReb":boxscore_playerstats["totReb"],
                                "assists":boxscore_playerstats["assists"],
                                "pFouls":boxscore_playerstats["pFouls"],
                                "steals":boxscore_playerstats["steals"],
                                "turnovers":boxscore_playerstats["totReb"],
                                "blocks":boxscore_playerstats["blocks"],
                                "plusMinus":boxscore_playerstats["plusMinus"],
                                "dnp":boxscore_playerstats["dnp"]})
    

    

In [39]:
boxscore_player_df=pd.DataFrame(boxscore_player_list)
boxscore_player_df.dtypes

assists      object
blocks       object
defReb       object
dnp          object
fga          object
fgm          object
fgp          object
fta          object
ftm          object
ftp          object
min          object
offReb       object
pFouls       object
personId     object
plusMinus    object
points       object
pos          object
steals       object
teamId       object
totReb       object
tpa          object
tpm          object
tpp          object
turnovers    object
dtype: object

In [16]:
#Load NBA_TEAMS table
# NBA_Schedule_DF_GameId.to_sql('NBA_SCHEDULE_GAME_DATE',conn, if_exists='replace', index=False)